In [1]:
import tensorflow as tf
import json
import numpy as np
import re
from tqdm import tqdm

2024-12-15 18:28:41.636054: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734280121.649611  122196 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734280121.653602  122196 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-15 18:28:41.669036: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
punctuation = re.compile(r"[^\w\s-]")
def preprocess_text(text):


    # text = contractions.fix(text) # expand contractions //stop now to save time annotating


    # text = spell_check(text) # spell check
    # words = text.split()
    # corrected_words = [spell.correction(word) or word for word in words]
    # text = ' '.join(corrected_words)


    text = tf.strings.strip(text) # leading/trailing whitespace removal


    text = tf.strings.lower(text)

    text = punctuation.sub("", text.numpy().decode('utf-8')) # punctuation removal


    # text = nlp(text) # lemmatization


    # text = tokenizer.tokenize(text.numpy().decode('utf-8'))  # tokenization

    return text

In [3]:
data_train = []
with open('PIZZA_train.json', 'r') as f:
    for line in f:
        data_train.append(json.loads(line))

print(len(data_train))

2456446


In [4]:
with open('PIZZA_dev.json', 'r') as file:
    data_dev = json.load(file)

# Print the loaded data
print(len(data_dev))

348


In [4]:
data_test = []
with open('PIZZA_test.json', 'r') as f:
    for line in f:
        data_test.append(json.loads(line))

print(len(data_test))

1357


In [5]:
def extract_text_train(data):
    return data['train.SRC'], data['train.EXR'], data['train.TOP'], data['train.TOP-DECOUPLED']

In [6]:
def extract_text_dev(data):
    return data['dev.SRC'], data['dev.EXR'], data['dev.TOP'], data['dev.PCFG_ERR']

In [5]:
def extract_text_test(data):
    return data['test.SRC'], data['test.EXR'], data['test.TOP'], data['test.PCFG_ERR']

In [7]:
sentences_train, explanations_train, topics_train, decoupled_topics_train = map(np.array, zip(*map(extract_text_train, data_train)))

In [8]:
sentences_dev, explanations_dev, topics_dev, decoupled_topics_dev = map(np.array, zip(*map(extract_text_dev, data_dev)))

In [6]:
sentences_test, explanations_test, topics_test, decoupled_topics_test = map(np.array, zip(*map(extract_text_test, data_test)))

In [9]:

processed_sentences_train = [
    preprocess_text(sentence) for sentence in tqdm(sentences_train, desc="Processing Train Sentences")
]
processed_sentences_train = np.array(processed_sentences_train)

Processing Train Sentences: 100%|██████████| 2456446/2456446 [05:03<00:00, 8093.37it/s]


In [10]:

processed_sentences_dev = [
    preprocess_text(sentence) for sentence in tqdm(sentences_dev, desc="Processing Dev Sentences")
]
processed_sentences_dev = np.array(processed_sentences_dev)

Processing Dev Sentences: 100%|██████████| 348/348 [00:00<00:00, 7822.43it/s]


In [7]:
processed_sentences_test = [
    preprocess_text(sentence) for sentence in tqdm(sentences_test, desc="Processing Test Sentences")
]
processed_sentences_test = np.array(processed_sentences_test)

Processing Test Sentences: 100%|██████████| 1357/1357 [00:01<00:00, 789.49it/s]


In [8]:
# np.save('processed_sentences_no_conct_train.npy', processed_sentences_train)
# np.save('processed_sentences_no_conct_dev.npy', processed_sentences_dev)
np.save('processed_sentences_no_conct_test.npy', processed_sentences_test)
